# 2.1 - seleccion de caracteristicas

In [2]:
# librerias

import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)

In [3]:
listings=pd.read_csv('../data/transform_data/listings.csv')

for c in listings.select_dtypes(include='int'):
    listings[c]=pd.to_numeric(listings[c], downcast='integer')

for c in listings.select_dtypes(include='float'):
    listings[c]=pd.to_numeric(listings[c], downcast='float')
    
listings.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21312 entries, 0 to 21311
Columns: 246 entries, id to Refrigerator
dtypes: float32(5), int16(10), int32(3), int8(226), object(2)
memory usage: 8.4 MB
